## Hackaton JobMadrid 21


In [40]:
import pandas as pd
import numpy as np
from scipy.stats import mode
import warnings
warnings.filterwarnings("ignore")

In [41]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error, mean_absolute_percentage_error

In [3]:
from sklearn.ensemble import VotingRegressor

In [4]:
train = pd.read_csv('Train_hack.csv')
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2010 entries, 0 to 2009
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Instagram  1985 non-null   float64
 1   Facebook   1971 non-null   float64
 2   Twitter    1956 non-null   float64
 3   Linkedin   1982 non-null   float64
 4   Discord    2000 non-null   float64
 5   Twitch     2010 non-null   float64
 6   Ingresos   2010 non-null   float64
dtypes: float64(7)
memory usage: 110.0 KB


In [5]:
bad_index = []

for i in range(len(train.Ingresos)):
    if train.Ingresos[i]< 0:
        bad_index.append(i)
bad_index

[1535]

In [6]:
train = train.drop([1535], axis=0)

In [7]:
train.head(10)

,Instagram,Facebook,Twitter,Linkedin,Discord,Twitch,Ingresos
0,11.257702,2.943174,6.803809,15.873606,5.545977,21.238601,269.800741
1,9.304230,2.709958,5.095496,13.467107,5.900347,18.869122,86.752487
2,NaN,4.936659,6.173092,14.461141,6.763816,22.107658,348.569975
3,9.423370,4.507282,7.762059,15.635129,7.707726,19.378033,415.976620
4,NaN,5.258533,5.760111,13.585080,6.574550,22.045628,346.167361
5,9.459845,4.155854,9.583170,13.794333,5.920768,21.118832,518.907019
6,10.769865,4.314324,6.161330,13.503831,7.096601,20.442303,305.012738
7,10.022232,5.160177,7.490064,14.454546,6.861220,20.931968,450.330442
8,9.877503,4.076456,6.354966,15.586167,7.446985,21.909247,291.453989
9,11.970156,5.576741,NaN,13.563709,7.363695,21.687212,403.785685


In [8]:
test = pd.read_csv('Test_predictors.csv')
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 990 entries, 0 to 989
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Instagram  990 non-null    float64
 1   Facebook   990 non-null    float64
 2   Twitter    990 non-null    float64
 3   Linkedin   990 non-null    float64
 4   Discord    990 non-null    float64
 5   Twitch     990 non-null    float64
dtypes: float64(6)
memory usage: 46.5 KB


In [9]:
nan_cols=train.isna().sum()
nan_cols

Instagram    25
Facebook     39
Twitter      54
Linkedin     28
Discord      10
Twitch        0
Ingresos      0
dtype: int64

In [10]:
sum(nan_cols)

156

In [11]:
train.describe()


,Instagram,Facebook,Twitter,Linkedin,Discord,Twitch,Ingresos
count,1984.000000,1970.000000,1955.000000,1981.000000,1999.000000,2009.000000,2009.000000
mean,9.970445,4.076864,6.707480,14.959887,7.033443,20.974221,316.754318
std,1.000638,1.010582,1.007790,0.981876,1.018451,0.994428,103.778646
min,6.796837,0.572794,2.673021,11.290172,3.387479,17.660218,3.048432
25%,9.324932,3.397869,5.988434,14.304278,6.353845,20.308660,244.107021
50%,9.961573,4.075472,6.720429,14.956465,7.056407,20.970163,316.423179
75%,10.608643,4.745237,7.408249,15.649198,7.737152,21.629633,387.624335
max,13.492470,7.205183,10.001590,18.670235,10.682078,24.323364,652.044392


In [12]:
insta = train.Instagram
insta


0       11.257702
1        9.304230
2             NaN
3        9.423370
4             NaN
          ...    
2005     8.020629
2006    10.607044
2007    10.084160
2008    10.952972
2009     8.637146
Name: Instagram, Length: 2009, dtype: float64

In [13]:
train.Instagram.median()

9.96157319

In [14]:
train.Instagram.mode()

0        6.796837
1        7.023952
2        7.025669
3        7.213555
4        7.224970
          ...    
1979    13.005687
1980    13.059283
1981    13.117004
1982    13.185245
1983    13.492470
Length: 1984, dtype: float64

In [15]:
# Revisar los valores perdidos
#Creamos una función:
def num_missing(x):
  return sum(x.isnull())

#Aplicamos por columna:
print ("Valores nulos por columna para train")
print (train.apply(num_missing, axis=0))
print ("--------------------------------------------")
print ("Valores nulos por columna para test")
print (test.apply(num_missing, axis=0))

Valores nulos por columna para train
Instagram    25
Facebook     39
Twitter      54
Linkedin     28
Discord      10
Twitch        0
Ingresos      0
dtype: int64
--------------------------------------------
Valores nulos por columna para test
Instagram    0
Facebook     0
Twitter      0
Linkedin     0
Discord      0
Twitch       0
dtype: int64


In [16]:
for column in ['Instagram', 'Facebook', 'Twitter', 'Linkedin','Discord']:
    train[column] = train[column].fillna(train[column].median())

In [17]:
train['Instagram'].fillna(train['Instagram'].median())

0       11.257702
1        9.304230
2        9.961573
3        9.423370
4        9.961573
          ...    
2005     8.020629
2006    10.607044
2007    10.084160
2008    10.952972
2009     8.637146
Name: Instagram, Length: 2009, dtype: float64

In [18]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2009 entries, 0 to 2009
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Instagram  2009 non-null   float64
 1   Facebook   2009 non-null   float64
 2   Twitter    2009 non-null   float64
 3   Linkedin   2009 non-null   float64
 4   Discord    2009 non-null   float64
 5   Twitch     2009 non-null   float64
 6   Ingresos   2009 non-null   float64
dtypes: float64(7)
memory usage: 125.6 KB


In [19]:
nan_cols=train.isna().sum()
nan_cols

Instagram    0
Facebook     0
Twitter      0
Linkedin     0
Discord      0
Twitch       0
Ingresos     0
dtype: int64

In [20]:
# Revisar los valores perdidos
#Creamos una función:
def num_missing(x):
  return sum(x.isnull())

#Aplicamos por columna:
print ("Valores nulos por columna para train")
print (train.apply(num_missing, axis=0))
print ("--------------------------------------------")
print ("Valores nulos por columna para test")
print (test.apply(num_missing, axis=0))

Valores nulos por columna para train
Instagram    0
Facebook     0
Twitter      0
Linkedin     0
Discord      0
Twitch       0
Ingresos     0
dtype: int64
--------------------------------------------
Valores nulos por columna para test
Instagram    0
Facebook     0
Twitter      0
Linkedin     0
Discord      0
Twitch       0
dtype: int64


In [21]:
train.describe()

,Instagram,Facebook,Twitter,Linkedin,Discord,Twitch,Ingresos
count,2009.000000,2009.000000,2009.000000,2009.000000,2009.000000,2009.000000,2009.000000
mean,9.970335,4.076836,6.707828,14.959839,7.033557,20.974221,316.754318
std,0.994390,1.000720,0.994149,0.975007,1.015913,0.994428,103.778646
min,6.796837,0.572794,2.673021,11.290172,3.387479,17.660218,3.048432
25%,9.331652,3.415120,6.016130,14.318979,6.358906,20.308660,244.107021
50%,9.961573,4.075472,6.720429,14.956465,7.056407,20.970163,316.423179
75%,10.601700,4.726999,7.394616,15.636826,7.734647,21.629633,387.624335
max,13.492470,7.205183,10.001590,18.670235,10.682078,24.323364,652.044392


In [22]:
stats=train.describe().T

stats['IQR']=stats['75%'] - stats['25%']

stats.tail()

,count,mean,std,min,25%,50%,75%,max,IQR
Twitter,2009.0,6.707828,0.994149,2.673021,6.016130,6.720429,7.394616,10.001590,1.378486
Linkedin,2009.0,14.959839,0.975007,11.290172,14.318979,14.956465,15.636826,18.670235,1.317847
Discord,2009.0,7.033557,1.015913,3.387479,6.358906,7.056407,7.734647,10.682078,1.375740
Twitch,2009.0,20.974221,0.994428,17.660218,20.308660,20.970163,21.629633,24.323364,1.320973
Ingresos,2009.0,316.754318,103.778646,3.048432,244.107021,316.423179,387.624335,652.044392,143.517314


In [23]:
#train.select_dtypes(inlude=['int64','float64']).columns.tolist()
outlier_list = train.select_dtypes(include=['int64','float64']).columns.tolist()
outlier_list

['Instagram',
 'Facebook',
 'Twitter',
 'Linkedin',
 'Discord',
 'Twitch',
 'Ingresos']

In [24]:
from collections import Counter

#drop the label 


def detect_outlier(df,outlier_list):
    
    outlier_indices = []
    
    for f in outlier_list:
        
        #lower quartile
        q1 = np.percentile(train[f],25)
        
        #upper quartile
        q3 = np.percentile(train[f],75)
        
        #interquartile range
        iqr = q3-q1
        
        #with coefficient
        with_coef = 1.5*iqr
        
        #detect outlier(s)
        lower_rule = q1-with_coef
        upper_rule = q3+with_coef
        
        outlier_variable = train[(train[f]<lower_rule) | (train[f]>upper_rule)].index
        outlier_indices.extend(outlier_variable)
    
    #converting to amount
    outlier_indices = Counter(outlier_indices)
    multiple_outliers = list(i for i,v in outlier_indices.items() if v>2)
    
    return multiple_outliers

 
#drop outliers
train = train.drop(detect_outlier(train,outlier_list),axis=0).reset_index(drop=True)

In [25]:
detect_outlier(train,outlier_list)

[]

In [26]:
X = train.drop(['Ingresos'], axis = 1)
y = train['Ingresos']

In [27]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2009 entries, 0 to 2008
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Instagram  2009 non-null   float64
 1   Facebook   2009 non-null   float64
 2   Twitter    2009 non-null   float64
 3   Linkedin   2009 non-null   float64
 4   Discord    2009 non-null   float64
 5   Twitch     2009 non-null   float64
dtypes: float64(6)
memory usage: 94.3 KB


In [42]:
pipelines = {
    'rf': make_pipeline(RandomForestRegressor(random_state=42)),
    'gb': make_pipeline(GradientBoostingRegressor(random_state=42)),
    'ridge': make_pipeline(StandardScaler(), Ridge(random_state=42)),
    'lasso': make_pipeline(StandardScaler(), Lasso(random_state=42)),
    'enet':make_pipeline(StandardScaler(), ElasticNet(random_state=42)),
    'lr': make_pipeline(StandardScaler(), LinearRegression()),
    'knn': make_pipeline(StandardScaler(), KNeighborsRegressor()),
    'svr': make_pipeline(StandardScaler(), SVR())}

In [43]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.20, random_state=42)

In [44]:
for key, value in pipelines.items():
    print(key, type(value))
    rf_hyperparameters = {
    'randomforestregressor__n_estimators': [100, 200,300,500,750],
    'randomforestregressor__max_features': ['auto', 0.3, 0.6]
    }

    gb_hyperparameters = {
    'gradientboostingregressor__n_estimators':[100, 200, 300,500,750],
    'gradientboostingregressor__learning_rate': [0.05, 0.1, 0.2],
    'gradientboostingregressor__max_depth':[1, 3, 5]
    }

    ridge_hyperparameters = {
        'ridge__alpha':[0.01, 0.05, 0.1, 1, 5]
    }

    lasso_hyperparameters = {
        'lasso__alpha':[0.01, 0.05, 0.1, 0.5, 1, 5]
    }

    enet_hyperparameters = {
        'elasticnet__alpha':[0.01, 0.05, 0.1, 1, 5],
        'elasticnet__l1_ratio':[0.1, 0.3, 0.5, 0.7, 0.9]
    }

    lr_hyperparameters = {
        'linearregression__n_jobs': [-1]
    }
    
    knn_hyperparameters = {
        "kneighborsregressor__n_neighbors": np.arange(1,50),
        "kneighborsregressor__weights": ["uniform","distance"]
    }
    svr_hyperparameters= {
    'svr__kernel':['linear', 'poly', 'rbf', 'sigmoid', 'precomputed'],
    'svr__degree':[3,5,7]
    }

rf <class 'sklearn.pipeline.Pipeline'>
gb <class 'sklearn.pipeline.Pipeline'>
ridge <class 'sklearn.pipeline.Pipeline'>
lasso <class 'sklearn.pipeline.Pipeline'>
enet <class 'sklearn.pipeline.Pipeline'>
lr <class 'sklearn.pipeline.Pipeline'>
knn <class 'sklearn.pipeline.Pipeline'>
svr <class 'sklearn.pipeline.Pipeline'>


In [45]:
hyperparameter_grids = {
    'rf': rf_hyperparameters,
    'gb': gb_hyperparameters,
    'ridge': ridge_hyperparameters,
    'lasso': lasso_hyperparameters,
    'enet': enet_hyperparameters,
    'lr': lr_hyperparameters,
    'knn': knn_hyperparameters,
    'svr': svr_hyperparameters
    
}

In [46]:
models = {}

for key in pipelines.keys():
    models[key]= GridSearchCV(pipelines[key],hyperparameter_grids[key], cv=5)
    
models.keys()

dict_keys(['rf', 'gb', 'ridge', 'lasso', 'enet', 'lr', 'knn', 'svr'])

In [47]:
best_estimators = []

for key in models.keys():
    clf = models[key].fit(X_train,y_train)
    best_estimators.append(clf.best_estimator_)
    print(key, "is trained and tuned")
    print('Method: {} Best: {}' .format(key,clf.best_estimator_))

rf is trained and tuned
Method: rf Best: Pipeline(steps=[('randomforestregressor',
                 RandomForestRegressor(n_estimators=750, random_state=42))])
gb is trained and tuned
Method: gb Best: Pipeline(steps=[('gradientboostingregressor',
                 GradientBoostingRegressor(learning_rate=0.05, max_depth=1,
                                           n_estimators=750,
                                           random_state=42))])
ridge is trained and tuned
Method: ridge Best: Pipeline(steps=[('standardscaler', StandardScaler()),
                ('ridge', Ridge(alpha=0.01, random_state=42))])
lasso is trained and tuned
Method: lasso Best: Pipeline(steps=[('standardscaler', StandardScaler()),
                ('lasso', Lasso(alpha=0.1, random_state=42))])
enet is trained and tuned
Method: enet Best: Pipeline(steps=[('standardscaler', StandardScaler()),
                ('elasticnet',
                 ElasticNet(alpha=0.01, l1_ratio=0.9, random_state=42))])
lr is trained and tu

In [48]:
for key in models:
    preds=models[key].predict(X_test)
    print(key)
    print("R squared: ", round(r2_score(y_test,preds),3))
    print('MAE: ', round(mean_absolute_error(y_test,preds),3))
    print("MSE score:", round(mean_squared_error(preds, y_test), 4))
    print("RMSE score:", round(np.sqrt(mean_squared_error(preds, y_test)), 4))
    print("MAPE score:", round(mean_absolute_percentage_error(preds, y_test), 4))
    print("--------------------")

rf
R squared:  0.976
MAE:  10.252
MSE score: 274.5114
RMSE score: 16.5684
MAPE score: 0.0375
--------------------
gb
R squared:  0.986
MAE:  6.74
MSE score: 159.5049
RMSE score: 12.6295
MAPE score: 0.0244
--------------------
ridge
R squared:  0.99
MAE:  2.664
MSE score: 118.2969
RMSE score: 10.8764
MAPE score: 0.0102
--------------------
lasso
R squared:  0.99
MAE:  2.612
MSE score: 117.9902
RMSE score: 10.8623
MAPE score: 0.0099
--------------------
enet
R squared:  0.99
MAE:  2.669
MSE score: 118.1772
RMSE score: 10.8709
MAPE score: 0.0101
--------------------
lr
R squared:  0.99
MAE:  2.664
MSE score: 118.2975
RMSE score: 10.8765
MAPE score: 0.0102
--------------------
knn
R squared:  0.922
MAE:  22.242
MSE score: 880.7535
RMSE score: 29.6775
MAPE score: 0.0789
--------------------
svr
R squared:  0.99
MAE:  1.92
MSE score: 117.1056
RMSE score: 10.8215
MAPE score: 0.0073
--------------------


In [51]:
df_test = pd.read_csv('Test_predictors.csv')

In [52]:
df_test['final_test'] = models['svr'].predict(df_test)

In [53]:
df_test

,Instagram,Facebook,Twitter,Linkedin,Discord,Twitch,final_test
0,10.527703,4.960205,6.479963,15.498262,6.955613,20.025227,374.912589
1,8.850377,2.512854,7.353186,14.727766,7.401905,18.968000,231.801975
2,9.544153,3.244153,7.148553,15.424961,6.664496,21.347135,283.466226
3,8.376256,3.885037,7.901298,14.386792,6.638287,21.202560,359.250695
4,9.730666,4.226696,6.541970,14.752687,6.498743,19.926721,310.717924
...,...,...,...,...,...,...,...
985,11.499436,4.602513,8.675953,15.946527,6.669920,19.062842,527.980558
986,9.560985,4.918870,6.874780,14.576312,7.219337,19.923273,380.358580
987,7.098913,5.244809,8.360054,14.273859,7.387039,21.476268,465.187563
988,10.162326,5.202366,7.109165,15.774028,8.147507,21.378660,431.788981


In [54]:
predicion = df_test['final_test']
predicion

0      374.912589
1      231.801975
2      283.466226
3      359.250695
4      310.717924
          ...    
985    527.980558
986    380.358580
987    465.187563
988    431.788981
989    388.165242
Name: final_test, Length: 990, dtype: float64

In [57]:
predicion.to_json('predicciones.json',orient='split', index=False)